In [2]:
import warnings
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import timesfm

#from ..lucas import forecast_utils
from forecast_utils import (
    generate_synthetic_financial_data,
    initialize_timesfm_model,
    generate_prediction_intervals,
    create_professional_forecast_visualization
)

 See https://github.com/google-research/timesfm/blob/master/README.md for updated APIs.
Loaded PyTorch TimesFM, likely because python version is 3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)].


In [27]:
feats = pd.read_csv('g:\Python Projects\Sapheneia\sapheneia\data\indicators3.csv')
#feats.head()
feats.replace('#VALUE!',np.nan,inplace=True)
corr = feats[feats.columns[1:]].corr()
display(corr.head())
highest_correlations = {}
for col in corr.columns:
    # Exclude self-correlation (value of 1) and find the maximum absolute value
    max_corr_value = corr[col].drop(col).abs().max()
    # Find the column name corresponding to this maximum absolute value
    highest_correlated_feature = corr[col].drop(col).abs().idxmax()
    highest_correlations[col] = (highest_correlated_feature, max_corr_value)

print("\nHighest Correlation for Each Column:")
for col, (feature, value) in highest_correlations.items():
    print(f"Column '{col}': Highest correlation with '{feature}' at {value:.4f}")

,Daily PL,WinningDays,LosingDays,Total BP,NetBetaDelta,DoDNetBeta,BetaIncreaseDays,BetaDecreaseDays,NetDelta,GrossDelta,...,Quarter,SPX OPEN,SPX HIGH,SPX LOW,INDU_Last,INDU OtO Returns,INDU OtC Returns,INDU OPEN,INDU HIGH,INDU LOW
Daily PL,1.000000,0.373798,-0.395682,0.092839,0.180425,0.110123,0.151512,-0.164980,0.182101,0.096903,...,0.024075,0.155921,0.166734,0.171006,0.146654,0.161104,0.207892,0.118140,0.132691,0.135246
WinningDays,0.373798,1.000000,-0.501893,-0.002259,0.091826,0.008696,0.044994,-0.096227,0.076007,-0.031870,...,0.016004,0.093287,0.097126,0.103654,0.080847,0.141053,0.136221,0.062898,0.068236,0.074963
LosingDays,-0.395682,-0.501893,1.000000,0.017467,-0.047120,-0.041790,-0.050374,0.074490,-0.038986,-0.037653,...,-0.086404,-0.002589,-0.009931,-0.012958,0.018695,-0.142561,-0.186434,0.043799,0.034620,0.028543
Total BP,0.092839,-0.002259,0.017467,1.000000,0.549330,0.040361,0.059356,-0.033892,0.531406,0.273604,...,-0.155019,0.543286,0.548766,0.536779,0.489652,-0.005847,0.017350,0.488089,0.500397,0.475445
NetBetaDelta,0.180425,0.091826,-0.047120,0.549330,1.000000,0.199653,0.204352,-0.225126,0.976184,0.387032,...,-0.009326,0.747156,0.755276,0.744789,0.614710,0.083023,0.081358,0.604819,0.622565,0.596366



Highest Correlation for Each Column:
Column 'Daily PL': Highest correlation with 'LosingDays' at 0.3957
Column 'WinningDays': Highest correlation with 'LosingDays' at 0.5019
Column 'LosingDays': Highest correlation with 'WinningDays' at 0.5019
Column 'Total BP': Highest correlation with 'SPX MOV_AVG_200D' at 0.5936
Column 'NetBetaDelta': Highest correlation with 'NetDelta' at 0.9762
Column 'DoDNetBeta': Highest correlation with 'BetaDecreaseDays' at 0.5567
Column 'BetaIncreaseDays': Highest correlation with 'BetaDecreaseDays' at 0.6087
Column 'BetaDecreaseDays': Highest correlation with 'BetaIncreaseDays' at 0.6087
Column 'NetDelta': Highest correlation with 'NetBetaDelta' at 0.9762
Column 'GrossDelta': Highest correlation with 'MTUM  PX_LAST' at 0.4623
Column 'SPX return': Highest correlation with 'VIX return' at 0.7126
Column 'SPXUpDays': Highest correlation with 'SPX INDX_ADV_NUM' at 0.4999
Column 'SPXDownDays': Highest correlation with 'SPXUpDays' at 0.4415
Column 'VIX return': Hi